In [ ]:
import requests
import json

In [ ]:
# Robust logging configuration
import logging
import sys

# Remove all handlers associated with the root logger object
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

LOG_FORMAT = '%(asctime)s - %(levelname)s - %(name)s - %(message)s'
LOG_LEVEL = logging.DEBUG

# Create handlers
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(LOG_LEVEL)
console_handler.setFormatter(logging.Formatter(LOG_FORMAT))


# Get root logger and set level
logger = logging.getLogger()
logger.setLevel(LOG_LEVEL)
logging.getLogger("requests").setLevel(logging.WARNING)


# Add handlers if not already present
if not any(isinstance(h, logging.StreamHandler) for h in logger.handlers):
    logger.addHandler(console_handler)

# Example usage:
logger.info('Logging is configured. Console and file output enabled.')

In [ ]:
UPSTREAM_REPO = "actions/runner-images"

In [21]:
releases_page = requests.get(
            f"https://api.github.com/repos/{UPSTREAM_REPO}/releases",
            params={'per_page': 1, 'page': 1}
        )

releases_page.json()


[{'url': 'https://api.github.com/repos/actions/runner-images/releases/235842698',
  'assets_url': 'https://api.github.com/repos/actions/runner-images/releases/235842698/assets',
  'upload_url': 'https://uploads.github.com/repos/actions/runner-images/releases/235842698/assets{?name,label}',
  'html_url': 'https://github.com/actions/runner-images/releases/tag/ubuntu24/20250728.1',
  'id': 235842698,
  'author': {'login': 'github-actions[bot]',
   'id': 41898282,
   'node_id': 'MDM6Qm90NDE4OTgyODI=',
   'avatar_url': 'https://avatars.githubusercontent.com/in/15368?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/github-actions%5Bbot%5D',
   'html_url': 'https://github.com/apps/github-actions',
   'followers_url': 'https://api.github.com/users/github-actions%5Bbot%5D/followers',
   'following_url': 'https://api.github.com/users/github-actions%5Bbot%5D/following{/other_user}',
   'gists_url': 'https://api.github.com/users/github-actions%5Bbot%5D/gists{/gist_id}',
   'star

In [22]:
def fetch_last_software_report(templates: list[str], repo: str = UPSTREAM_REPO) -> dict[str, dict]:
    """
    Fetches latest Runner Images software reports (internal.{os}.json) from the upstream repository.

    Args:
        templates (list[str]): List of templates to get software report internal json for.
        repo (str): The GitHub repository to fetch the reports from.
    Returns:
        dict[str, dict]: A dictionary where keys are template names and values are the contents
                         of the software report for that template.
    Example:
        >>> fetch_last_software_report(['win25', 'ubuntu24'])
        {
            'win25': {"NodeType": "HeaderNode", "Title": "Windows Server 2025", ...},
            'ubuntu24': {"NodeType": "HeaderNode", "Title": "Ubuntu 24.04", ...}
        }
    """

    reports = {}
    tags = []
    page = 1
    templates = templates.copy()  # Avoid mutating input

    while templates:
        try:
            releases_page = requests.get(
                f"https://api.github.com/repos/{repo}/releases",
                params={'per_page': 100, 'page': page}
            )
            releases_page.raise_for_status()
        except requests.exceptions.RequestException as e:
            logging.error(f"Error fetching releases for templates {templates}: {e}")
            break

        releases = releases_page.json()
        if not releases:
            break  # No more pages
        for release in releases:
            tag_name = release.get('tag_name', '')
            template_key = tag_name.split('/')[0]
            if template_key in templates:
                tags.append(tag_name)

                try:
                    for asset in release.get('assets', []):
                        if 'internal' in asset['name']:
                            response = requests.get(asset['browser_download_url'])
                            response.raise_for_status()
                            reports[template_key] = response.json()
                            logging.debug(f"Fetched report for {template_key}")
                except requests.exceptions.RequestException as e:
                    logging.error(f"Error fetching asset for {template_key}: {e}")
                    continue

                templates.remove(template_key)
        logging.debug(f"Found tags: {tags}, page: {page}, templates left: {templates}")
        page += 1



    return reports


In [23]:
fetch_last_software_report(['win25', 'ubuntu24'])

2025-07-31 18:03:04,409 - DEBUG - root - Fetched report for ubuntu24
2025-07-31 18:03:05,024 - DEBUG - root - Fetched report for win25
2025-07-31 18:03:05,024 - DEBUG - root - Found tags: ['ubuntu24/20250728.1', 'win25/20250727.1'], page: 1, templates left: []
2025-07-31 18:03:05,024 - DEBUG - root - Fetched report for win25
2025-07-31 18:03:05,024 - DEBUG - root - Found tags: ['ubuntu24/20250728.1', 'win25/20250727.1'], page: 1, templates left: []


{'ubuntu24': {'NodeType': 'HeaderNode',
  'Title': 'Ubuntu 24.04',
  'Children': [{'NodeType': 'ToolVersionNode',
    'ToolName': 'OS Version:',
    'Version': '24.04.2 LTS'},
   {'NodeType': 'ToolVersionNode',
    'ToolName': 'Kernel Version:',
    'Version': '6.11.0-1018-azure'},
   {'NodeType': 'ToolVersionNode',
    'ToolName': 'Image Version:',
    'Version': '20250728.1.0'},
   {'NodeType': 'ToolVersionNode',
    'ToolName': 'Systemd version:',
    'Version': '255.4-1ubuntu8.10'},
   {'NodeType': 'HeaderNode',
    'Title': 'Installed Software',
    'Children': [{'NodeType': 'HeaderNode',
      'Title': 'Language and Runtime',
      'Children': [{'NodeType': 'ToolVersionNode',
        'ToolName': 'Bash',
        'Version': '5.2.21(1)-release'},
       {'NodeType': 'ToolVersionsListNode',
        'ToolName': 'Clang',
        'Versions': ['16.0.6', '17.0.6', '18.1.3'],
        'MajorVersionRegex': '^\\d+',
        'ListType': 'Inline'},
       {'NodeType': 'ToolVersionsListNode',
  